In [4]:
import numpy as np
import pandas as pd
import re 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [2]:
all_data = pd.read_csv('all_novelty.csv')

In [5]:
x = 0
list_n = []
while x < len(all_data):
    s = all_data.iloc[x]
    clean = s['lyric_vector'].split(' ')
    arr = []
    for v in clean:
        v = re.sub(r"\[", "", v)
        v = re.sub(r"\\n", "", v)
        v = re.sub(r"\]", "", v)
        if v =='':
            pass
        else:
            v = float(v)
            arr.append(v)
    vec = np.asarray(arr)
    list_n.append(vec)
    x +=1
    
all_data['lyric_vector'] = list_n

In [6]:
all_data['Decade'] = round(all_data['Year'],-1)

In [7]:
Macc_df = pd.DataFrame(columns = ['Rock','Country','Pop','Rap','R&B','Electronica'])
Lacc_df = pd.DataFrame(columns = ['Rock','Country','Pop','Rap','R&B','Electronica'])

In [ ]:
g_list = ['Rock','Country','Pop','Rap','R&B','Electronica']
i = 0
while i < 6:
    genre = g_list[i]
    g = all_data[all_data['allmusic_genre']=='Rock'][['allmusic_genre','Decade','key', 'energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'mode',
       'time_signature', 'loudness', 'valence', 'danceability', 'time_sec','lyric_vector']]
    X = g[['key', 'energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'mode',
       'time_signature', 'loudness', 'valence', 'danceability', 'time_sec','lyric_vector']]
    y = g['Decade']

    rf_inst = RandomForestClassifier(n_estimators = 500, random_state=np.random.RandomState(0))
    cv = RepeatedKFold(n_splits=5, n_repeats=5)
    m = cross_val_score(rf_inst, X[['key', 'energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'mode',
           'time_signature', 'loudness', 'valence', 'danceability', 'time_sec']], y, cv = cv)
    l = cross_val_score(rf_inst,pd.DataFrame(np.column_stack(list(zip(*X['lyric_vector'])))),y , cv = cv)
    Macc_df[genre] = m
    Lacc_df[genre] = l
    i+=1

In [ ]:
fig, ax = plt.subplots(6,figsize=(6,8))
fig.tight_layout(h_pad = 3.5)

g_list = ['Rock','R&B','Rap','Country','Pop','Electronica']
i = 0
while i < 6:
    genre = g_list[i]
    ax[i].boxplot([Macc_df[genre],Lacc_df[genre]], vert = False, labels = ['MIR Features','Lyric Features'])
    ax[i].set_title(genre)
    ax[i].set_xlabel('Model Accuracy')
    ax[i].set_xlim((.4,.61))
    i+=1
    
plt.savefig('RF_acc',bbox_inches="tight")